In [2]:
!pip install pygame
import pygame
import time

# tracking stuff
!pip install  opencv-python mediapipe sklearn matplotlib 
#tensorflow-gpu==2.4.1
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# sound stuff
!pip install pyaudio
!pip install pysinewave
import time
import pyaudio
from pysinewave import SineWave
import pyaudio as p
import math
import threading

mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_landmarks(image, results):
    # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    # mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

def draw_styled_landmarks(image, results):
    # # Draw face connections
    # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
    #                          mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
    #                          mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
    #                          ) 
    # # Draw pose connections
    # mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
    #                          mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
    #                          mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
    #                          ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

def extract_keypoints(results):
    # face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh, pose])

# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
# actions = np.array(['hello', 'thanks', 'iloveyou'])
actions = np.array(['big_boy_fist_right', 'big_boy_index_right', 'big_hand_right', 'big_boy_fist_left', 'big_boy_index_left', 'big_hand_left', 'nothing'])

# Thirty videos worth of data
no_sequences = 200

# Videos are going to be 30 frames in length
sequence_length = 5

# Folder start
# start_folder = 30

for action in actions: 
    # dirmax = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))
    for sequence in range(0,no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass



from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

label_map = {label:num for num, label in enumerate(actions)}

sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(5,258)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

model.load_weights('big_boy_data.h5')

from scipy import stats



pygame 2.3.0 (SDL 2.24.2, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


sine wave

In [3]:
def pitch_to_freq(pitch):
    return 440 * 2 ^ ((pitch-9)/12)

def freq_to_pitch(freq):
    return 12 * math.log2(freq/440) + 9

# CHORDS = {
#     0   : [-10, -6, -4],
#     0.1 : [-7, -4, -2],
#     0.2 : [-5, -2, 1],
#     0.3 : [-3, 1, 3],
#     0.4 : [-1, 3, 5],
#     0.5 : [0, 4, 7],
#     0.6 : [2, 5, 9],
#     0.7 : [3, 7, 10],
#     0.8 : [5, 9, 12],
#     0.9 : [7, 11, 14],
#     1   : [9, 13, 16]
# }

CHORDS = {
    0   : [-6, -2, 1],
    0.1 : [-5, -1, 2],
    0.2 : [-4, 0, 3],
    0.3 : [-2, 2, 5],
    0.4 : [-1, 3, 6],
    0.5 : [0, 4, 7],
    0.6 : [0.5, 5, 8],
    0.7 : [2, 6, 9],
    0.8 : [4, 8, 11],
    0.9 : [5, 9, 12],
    1   : [6, 10, 13]
}



class Waves:
    def __init__(self):
        self.wave1 = SineWave(pitch=0, pitch_per_second=100, decibels_per_second=100)
        self.wave2 = SineWave(pitch=4, pitch_per_second=100, decibels_per_second=100)
        self.wave3 = SineWave(pitch=7, pitch_per_second=100, decibels_per_second=100)
        self.waves = [self.wave1, self.wave2, self.wave3]

    def get_chords(self, x_pos):
        chord = [0, 4, 7]
        offset = 2
        difference = x_pos - 0.5

        new_chord = []
        for c in chord:
            new_chord.append(c + (difference*10) + offset)

        return new_chord

    def update_chord(self, x_pos):
        pitches = CHORDS[x_pos]
        for i in range(3):
            self.waves[i].set_pitch(pitches[i])

        self.run()

    def set_volume(self, volume):
        for wave in self.waves:
            wave.set_volume(volume)

    def run(self):
        for wave in self.waves:
            wave.play()
        # time.sleep(1.5) # Comment out when done

    def stop(self):
        for wave in self.waves:
            wave.stop()

pygame

In [12]:
# Initialize Pygame
pygame.mixer.init()

# Set the number of mixer channels
pygame.mixer.set_num_channels(3)

# Load the sound files
sound1 = pygame.mixer.Sound('high-hat.wav')
sound2 = pygame.mixer.Sound('bass.wav')
sound3 = pygame.mixer.Sound('synth-chord.wav')

sound1.set_volume(10)
sound2.set_volume(10)
sound3.set_volume(10)

print(sound1.get_length())

channel1 = pygame.mixer.Channel(0)
channel2 = pygame.mixer.Channel(1)
channel3 = pygame.mixer.Channel(1)

# Define a function to play sounds based on input
def play_sound(input):
    if input == 1 and not channel1.get_busy():
        # channel = pygame.mixer.Channel(0)
        channel1.play(sound1)
        channel1.fadeout(int(sound1.get_length()*1000 - 2500))
    elif input == 2 and not channel2.get_busy():
        # channel = pygame.mixer.Channel(1)
        channel2.play(sound2)
        # channel2.fadeout(int(sound2.get_length()*1000 - 30))
    elif input == 3 and not channel3.get_busy():
        # channel = pygame.mixer.Channel(2)
        channel3.play(sound3)
        # channel3.fadeout(int(sound3.get_length()*1000 - 30))


8.907479286193848


In [5]:
play_sound(3)

In [14]:

# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)

sine_wave = Waves()
sine_wave.run()

y_pos = 0
x_pos = 0
sequence_length = 5

count = 0
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        # print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-sequence_length:]
        
        if len(sequence) == sequence_length:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            predictions.append(np.argmax(res))
            
            
        # sinth sounds
        if results.right_hand_landmarks != None:
            right_pos = results.right_hand_landmarks.landmark[0]
            y_pos = right_pos.y
        if results.right_hand_landmarks != None:
            right_pos = results.right_hand_landmarks.landmark[0]
            x_pos = right_pos.x

        # range 0 to 50
        y_pos = 1 - y_pos
        volume = y_pos
        sine_wave.set_volume(volume)

        # range -10 to 10
        x_pos = 1 - x_pos
        x_pos = round(x_pos, 1)
        sine_wave.update_chord(x_pos)

        
        ## play loop
        left_hand = res[:3]
        right_hand = res[3:6]
        
        guess = np.argmax(res)
        if guess != 6:

            guess_left = np.argmax(left_hand)
            guess_right = np.argmax(right_hand)

            left_accuracy = left_hand[guess_left]
            right_accuracy = right_hand[guess_right]

            if left_accuracy > 0.9:
                if guess_left == 0:
                    play_sound(1)

                if guess_left == 1:
                    play_sound(2)

            if right_accuracy > 0.9:
                if guess_right == 0:
                    play_sound(3)

                # if guess_right == 1:
                #     play_sound(2)
        
        
        
        # # Show to screen
        cv2.imshow('OpenCV Feed', image)
        count += 1
        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    sine_wave.stop()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 18ms/step


In [8]:
cap.release()